In [ ]:
import cv2
import numpy as np
from skimage.morphology import skeletonize
from google.colab.patches import cv2_imshow

In [ ]:
# Define a function to classify neighborhood pixels into direct and diagonal
def classify_neighbors(x, y, img):
    direct_pixels = []
    diagonal_pixels = []

    for i in range(x - 1, x + 2):
        for j in range(y - 1, y + 2):
            if (i == x and j == y) or i < 0 or j < 0 or i >= img.shape[1] or j >= img.shape[0]:
                continue

            if img[j, i] == 255:
                if i == x or j == y:
                    direct_pixels.append((i, j))
                else:
                    diagonal_pixels.append((i, j))

    return direct_pixels, diagonal_pixels

# Define a function to extract individual components
def extract_components(img, min_size_threshold,endpoints, intersection_points):
    components = []
    component_id = 1

    #endpoints, intersection_points = identify_endpoints_and_intersections(img.copy())
    while endpoints or intersection_points:
        if endpoints:
            current_pixel = endpoints.pop()
        else:
            current_pixel = intersection_points.pop()

        stack = [current_pixel]
        component = set()

        while stack:
            x, y = stack.pop()
            component.add((x, y))
            img[y, x] = 0

            direct_neighbors, diagonal_neighbors = classify_neighbors(x, y, img)

            unvisited_neighbors = [
                (nx, ny) for nx, ny in direct_neighbors + diagonal_neighbors if img[ny, nx] == 255
            ]

            if len(unvisited_neighbors) == 1:
                stack.append(unvisited_neighbors[0])

        if len(component) >= min_size_threshold:
            components.append(component)

    return components

# Define a function to compute the equation of a line given two points
def compute_line_equation(point1, point2):
    x1, y1 = point1
    x2, y2 = point2

    a = y1 - y2
    b = x2 - x1
    c = x1 * y2 - x2 * y1

    return a, b, c

# Define a function to calculate perpendicular distances
def calculate_perpendicular_distances(component, line_equation):
    distances = []

    a, b, c = line_equation

    for x, y in component:
        distance = abs(a * x + b * y + c) / np.sqrt(a ** 2 + b ** 2)
        distances.append(distance)

    return distances

# Define a function to decode shape symbols
def decode_shape_symbols(distances):
    shape_symbols = []

    for i in range(len(distances) - 1):
        di = distances[i]
        dj = distances[i + 1]

        if np.isclose(di, dj):
            shape_symbols.append("E")  # Equal distances
        elif di < dj:
            shape_symbols.append("U")  # First distance is less
        else:
            shape_symbols.append("D")  # First distance is greater

    return shape_symbols

# Define a function to extract shape descriptors for a component
def extract_shape_descriptor(component):
    if len(component) < 2:
        return None  # Cannot extract shape descriptor for very small components

    # Convert the component set to a list
    component = list(component)

    # Get extreme end pixel coordinates
    p1 = component[0]
    p2 = component[-1]

    # Compute the line equation between P1 and P2
    line_equation = compute_line_equation(p1, p2)

    # Calculate perpendicular distances
    distances = calculate_perpendicular_distances(component, line_equation)

    # Decode shape symbols
    shape_symbols = decode_shape_symbols(distances)

    return shape_symbols
# Define a function to extract shape descriptors for all components
def extract_shape_descriptors(components):
    shape_descriptors = []

    for component in components:
        shape_descriptor = extract_shape_descriptor(component)
        if shape_descriptor is not None:
            shape_descriptors.append(shape_descriptor)

    return shape_descriptors



In [ ]:
def sgpd_string(filepath):
  # Load the grayscale character image
  character_image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

  # Threshold the image to create a binary image
  _, binary_image = cv2.threshold(character_image, 128, 255, cv2.THRESH_BINARY)

  # resizing the character_image
  binary_image = cv2.resize(binary_image, (40, 60))

  # Complement the binary image (invert pixel values)
  binary_image = 255 - binary_image

  # Apply morphological skeletonization
  skeleton = skeletonize(binary_image / 255)  # Divide by 255 to ensure 0/1 values

  cv2.imwrite('skeleton1.png', (skeleton * 255).astype(np.uint8))

  # Load the skeletonized image (binary image with skeleton)
  skeleton_image = cv2.imread('/content/skeleton1.png', cv2.IMREAD_GRAYSCALE)

  # Define lists to store endpoints and intersection points
  endpoints = []
  intersection_points = []

  # Iterate through the skeletonized image
  for y in range(1, skeleton_image.shape[0] - 1):
    for x in range(1, skeleton_image.shape[1] - 1):
      # Check if the current pixel is part of the skeleton
      if skeleton_image[y, x] == 255:
        # Define neighborhood pixel coordinates
        neighbors = [
            (y - 1, x - 1), (y - 1, x), (y - 1, x + 1),
            (y, x - 1), (y, x + 1),
            (y + 1, x - 1), (y + 1, x), (y + 1, x + 1)
        ]

        # Count the number of neighboring pixels that are part of the skeleton
        skeleton_neighbors = [skeleton_image[ny, nx] for ny, nx in neighbors]
        skeleton_count = sum(1 for pixel_value in skeleton_neighbors if pixel_value == 255)

        # Check for endpoints and intersection points based on criteria
        if skeleton_count == 1:
          endpoints.append((x, y))
        elif skeleton_count >= 3:
          intersection_points.append((x, y))


  # Set a minimum size threshold for components (e.g., 5 pixels)
  min_size_threshold = 5

  # Extract and filter components
  filtered_components = extract_components(skeleton_image, min_size_threshold, endpoints, intersection_points)

  # Extract shape descriptors for all components
  shape_descriptors = extract_shape_descriptors(filtered_components)

  # Combine shape descriptors to form SGPD
  sgpd = "".join("".join(sd) for sd in shape_descriptors)

  return sgpd


In [ ]:
# example
img = '/content/10.jpg'
print(sgpd_string(img))

EUUDDUUDDEEUDUDUD


In [ ]:
!unzip Train.zip -d /content/train  # Replace 'file.zip' and '/content/dataset_folder' with appropriate file names and paths

In [ ]:
import os
folder_path1 = '/content/train/Train'
sub_folders = os.listdir(folder_path1)
for folder in sub_folders:
  folder_path = os.path.join(folder_path1, folder)
  image_files = os.listdir(folder_path)
  file_path = '/content/'+folder+'.txt'
  # Iterate through each file in the folder
  for img_file in image_files:
    if img_file.endswith('.jpg') or img_file.endswith('.png'):  # Check for image extensions
        img_path = os.path.join(folder_path, img_file)

        str1 = sgpd_string(img_path)
        with open(file_path, 'a') as file:
          # Write data to the file
          # file.write(folder + ' -> '+str1+'\n')
          file.write(folder + ' '+str1+'\n')

In [ ]:
import os
from google.colab import files
folders = os.listdir(folder_path1)
for i in range(1,27):
  file_path = '/content/'+str(chr(ord('A')+i-1))+'.txt'
  files.download(file_path)
  # os.remove(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import shutil

# Specify the path of the folder you want to delete
folder_path = "/content/train_dataset"

# Check if the folder exists before attempting to delete
if os.path.exists(folder_path):
    # Delete the folder and its contents
    shutil.rmtree(folder_path)
    print(f"Folder '{folder_path}' deleted successfully.")
else:
    print(f"Folder '{folder_path}' does not exist.")

Folder '/content/train_dataset' deleted successfully.
